# Python FFI API Demo - jpegexp-rs

## Overview
This notebook demonstrates how to use the FFI (Foreign Function Interface) API of `jpegexp-rs` from Python to encode and decode JPEG and JPEG2000 images. It includes usage examples, error handling, and comparisons with standard libraries.

## Purpose
- Demonstrate Python FFI bindings for jpegexp-rs
- Show encoding/decoding workflows
- Compare results with Pillow and other standard libraries
- Test error handling with corrupted files

## Contents
1. Setup and requirements
2. FFI wrapper implementation
3. Basic encoding/decoding examples
4. Error handling demonstrations
5. Visual and quantitative comparisons

---

## Requirements

### Prerequisites
- **Compiled DLL**: The `jpegexp_rs.dll` binary must be compiled and available in either:
  - `target/release/jpegexp_rs.dll` (release build)
  - `target/debug/jpegexp_rs.dll` (debug build)

### Python Dependencies
Install the following Python packages:
- `numpy` - For array operations
- `matplotlib` - For image visualization
- `pillow` (PIL) - For image I/O and comparison

### Installation
```bash
pip install numpy matplotlib pillow
```


In [ ]:
# Standard library imports
import os

# Third-party imports for image processing and visualization
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# Utility function to generate test gradient images
# Creates a horizontal grayscale gradient from black (0) to white (255)

def generate_gradient(width, height):
    """
    Generate a horizontal grayscale gradient test image.

    Args:
        width: Image width in pixels
        height: Image height in pixels

    Returns:
        numpy.ndarray: 2D array of uint8 values (0-255) representing the gradient
    """
    arr = np.linspace(0, 255, width, dtype=np.uint8)
    grad = np.tile(arr, (height, 1))
    return grad

In [ ]:
# ============================================================================
# FFI Wrapper Implementation for jpegexp-rs
# ============================================================================
# This section implements a Python wrapper around the jpegexp-rs C FFI API
# using ctypes. It provides a high-level interface for encoding and decoding
# JPEG/JPEG2000 images from Python.
# ============================================================================

import ctypes
from ctypes import POINTER, c_void_p, c_int, c_uint32, c_size_t, c_ubyte

# Buscar DLL
possible_dlls = [
    os.path.join("..", "target", "release", "jpegexp_rs.dll"),
    os.path.join("target", "release", "jpegexp_rs.dll"),
    os.path.join("..", "target", "debug", "jpegexp_rs.dll")
]
DLL_PATH = None
for p in possible_dlls:
    if os.path.exists(p):
        DLL_PATH = os.path.abspath(p)
        break
if DLL_PATH:
    print(f"Usando DLL: {DLL_PATH}")
else:
    print("ADVERTENCIA: No se encontró DLL. Las pruebas FFI fallarán.")

class JpegExpFFI:
    def __init__(self, dll_path):
        if not os.path.exists(dll_path):
            raise RuntimeError(f"DLL no encontrada en {dll_path}")
        self.lib = ctypes.CDLL(dll_path)
        # Tipos
        self.lib.jpegexp_decoder_new.argtypes = [POINTER(c_ubyte), c_size_t]
        self.lib.jpegexp_decoder_new.restype = c_void_p
        self.lib.jpegexp_decoder_free.argtypes = [c_void_p]
        self.lib.jpegexp_decoder_free.restype = None
        class ImageInfo(ctypes.Structure):
            _fields_ = [("width", c_uint32),
                        ("height", c_uint32),
                        ("components", c_uint32),
                        ("bits_per_sample", c_uint32)]
        self.lib.jpegexp_decoder_read_header.argtypes = [c_void_p, POINTER(ImageInfo)]
        self.lib.jpegexp_decoder_read_header.restype = c_int
        self.lib.jpegexp_decoder_decode.argtypes = [c_void_p, POINTER(c_ubyte), c_size_t]
        self.lib.jpegexp_decoder_decode.restype = c_int
        self.lib.jpegexp_encode_jpeg.argtypes = [
            POINTER(c_ubyte), c_uint32, c_uint32, c_uint32,
            POINTER(c_ubyte), c_size_t, POINTER(c_size_t)
        ]
        self.lib.jpegexp_encode_jpeg.restype = c_int
        self.lib.jpegexp_encode_jpegls.argtypes = [
            POINTER(c_ubyte), c_uint32, c_uint32, c_uint32,
            POINTER(c_ubyte), c_size_t, POINTER(c_size_t)
        ]
        self.lib.jpegexp_encode_jpegls.restype = c_int
        self.ImageInfo = ImageInfo
    def decode(self, data):
        data_bytes = (c_ubyte * len(data)).from_buffer_copy(data)
        decoder = self.lib.jpegexp_decoder_new(data_bytes, len(data))
        if not decoder:
            raise RuntimeError("No se pudo crear el decodificador")
        try:
            info = self.ImageInfo()
            res = self.lib.jpegexp_decoder_read_header(decoder, ctypes.byref(info))
            if res != 0:
                raise RuntimeError(f"Fallo al leer cabecera: {res}")
            required_size = info.width * info.height * info.components
            output = (c_ubyte * required_size)()
            res = self.lib.jpegexp_decoder_decode(decoder, output, required_size)
            if res != 0:
                 raise RuntimeError(f"Fallo al decodificar: {res}")
            return bytes(output), info.width, info.height, info.components
        finally:
             self.lib.jpegexp_decoder_free(decoder)
    def encode_jpeg(self, pixels, width, height, components):
         pixel_bytes = (c_ubyte * len(pixels)).from_buffer_copy(pixels)
         out_size = len(pixels) * 2 + 1024
         output = (c_ubyte * out_size)()
         written = c_size_t(0)
         res = self.lib.jpegexp_encode_jpeg(
             pixel_bytes, width, height, components,
             output, out_size, ctypes.byref(written)
         )
         if res != 0:
             raise RuntimeError(f"Fallo al codificar JPEG: {res}")
         return bytes(output[:written.value])

## Example 1: Basic Encoding and Decoding

This example demonstrates the basic workflow:
1. Generate a test gradient image
2. Encode it to JPEG using the FFI API
3. Verify the encoded file with Pillow
4. Decode it back using the FFI API
5. Visualize the results

In [ ]:
# Basic encoding/decoding demonstration using the FFI API
# This cell shows the complete workflow from encoding to decoding

if DLL_PATH:
    try:
        print("Inicializando FFI...")
        ffi = JpegExpFFI(DLL_PATH)
        # Codificar gradiente
        print("Codificando gradiente en JPEG...")
        w, h = 256, 256
        grad = generate_gradient(w, h)
        grad_bytes = grad.tobytes()
        encoded_jpeg_bytes = ffi.encode_jpeg(grad_bytes, w, h, 1)
        print(f"JPEG codificado: {len(encoded_jpeg_bytes)} bytes.")
        # Verificar con Pillow
        from io import BytesIO
        img = Image.open(BytesIO(encoded_jpeg_bytes))
        print(f"Pillow abrió JPEG: {img.size} {img.mode}")
        plt.imshow(img, cmap='gray')
        plt.title("JPEG codificado por FFI")
        plt.show()
        # Decodificar usando FFI
        print("Decodificando JPEG...")
        decoded_pixels, dw, dh, dc = ffi.decode(encoded_jpeg_bytes)
        print(f"Decodificado: {dw}x{dh}, {dc} componentes.")
        dec_arr = np.frombuffer(decoded_pixels, dtype=np.uint8).reshape((dh, dw))
        plt.imshow(dec_arr, cmap='gray')
        plt.title("Pixels decodificados por FFI")
        plt.show()
    except Exception as e:
        print(f"Error en la demo FFI: {e}")
else:
    print("Saltando demo FFI (DLL no encontrada)")

## Error Handling and Edge Cases

### Error Scenarios
The FFI API can raise exceptions in several scenarios:
- **DLL not available**: When the compiled DLL cannot be found or loaded
- **Corrupt data**: When attempting to decode invalid or corrupted image files
- **Internal errors**: When encoding/decoding operations fail due to invalid parameters

### Best Practices
- Always wrap FFI calls in `try/except` blocks
- Validate input data before processing
- Check for DLL availability before attempting operations
- Handle specific error codes returned by the FFI functions

## Comparison with Standard Libraries

### Purpose
Compare decoding results from `jpegexp-rs` with standard Python libraries to:
- Validate correctness of the implementation
- Verify compatibility with standard formats
- Assess visual quality differences

### Libraries Used
- **Pillow (PIL)**: Standard Python imaging library
- **imagecodecs**: Advanced codec library with OpenJPEG support (optional)

### Comparison Methods
- Visual inspection (side-by-side images)
- Quantitative metrics (PSNR - Peak Signal-to-Noise Ratio)

In [ ]:
# Comparison example: Generate JPEG using Pillow for baseline comparison
# This creates a reference JPEG that we can compare against jpegexp-rs output

img_pillow = Image.fromarray(generate_gradient(256, 256))
img_pillow.save("grad_pillow.jpg", format="JPEG")
img_loaded = Image.open("grad_pillow.jpg")
plt.imshow(img_loaded, cmap='gray')
plt.title("JPEG generado por Pillow")
plt.show()

## Example 2: JPEG2000 Decoding

### Purpose
Demonstrate decoding of JPEG2000 (.jp2) files using the FFI API.

### Requirements
- A valid JPEG2000 test file in the specified path
- The file should be in the test_images directory structure

### Workflow
1. Load JPEG2000 file from disk
2. Decode using FFI API
3. Convert to numpy array
4. Visualize result (grayscale or color depending on components)

In [ ]:
# Decode a JPEG2000 file using the FFI API
# NOTE: Update the path to point to an actual JPEG2000 test file in your test_images directory
jp2_path = "../tests/artifacts_comprehensive/test_images/JPEG2000/CT1.JP2"
if DLL_PATH and os.path.exists(jp2_path):
    with open(jp2_path, "rb") as f:
        jp2_bytes = f.read()
    try:
        decoded_pixels, w, h, c = ffi.decode(jp2_bytes)
        arr = np.frombuffer(decoded_pixels, dtype=np.uint8).reshape((h, w, c) if c > 1 else (h, w))
        plt.imshow(arr, cmap='gray' if c == 1 else None)
        plt.title("JPEG2000 decodificado por FFI")
        plt.axis('off')
        plt.show()
    except Exception as e:
        print(f"Error al decodificar JPEG2000: {e}")
else:
    print("No se encontró el archivo JPEG2000 o la DLL.")

## Example 3: Error Handling with Corrupted Files

### Purpose
Demonstrate how the FFI API handles corrupted or invalid JPEG2000 files.

### Expected Behavior
- The API should detect corruption and raise appropriate exceptions
- Error messages should provide useful diagnostic information
- The decoder should fail gracefully without crashing

### Testing
Use a known problematic file (e.g., one that has caused errors in previous tests) to verify error handling.

In [ ]:
# Attempt to decode a corrupted JPEG2000 file
# This should trigger error handling and demonstrate exception behavior
# NOTE: Update the path to point to a known problematic file
corrupt_path = "../tests/artifacts_comprehensive/test_images/JPEG2000/bitwiser-icc-corrupted-tagcount-1999.jp2"
if DLL_PATH and os.path.exists(corrupt_path):
    with open(corrupt_path, "rb") as f:
        corrupt_bytes = f.read()
    try:
        decoded_pixels, w, h, c = ffi.decode(corrupt_bytes)
        arr = np.frombuffer(decoded_pixels, dtype=np.uint8).reshape((h, w, c) if c > 1 else (h, w))
        plt.imshow(arr, cmap='gray' if c == 1 else None)
        plt.title("JPEG2000 corrupto decodificado (¡debería fallar!)")
        plt.axis('off')
        plt.show()
    except Exception as e:
        print(f"Error esperado al decodificar archivo corrupto: {e}")
else:
    print("No se encontró el archivo corrupto o la DLL.")

## Example 4: RGB Image Encoding

### Purpose
Demonstrate encoding of RGB (3-component) images using the FFI API.

### Test Image
- Creates a synthetic RGB gradient image
- Red channel: horizontal gradient (0-255)
- Green channel: reverse horizontal gradient (255-0)
- Blue channel: constant value (128)

### Workflow
1. Generate RGB test image in numpy
2. Encode to JPEG using FFI API
3. Verify with Pillow
4. Visualize the result

In [ ]:
# Encode an RGB image using the FFI API
if DLL_PATH:
    try:
        # Create synthetic RGB test image with gradient patterns
        w, h = 128, 128
        rgb = np.zeros((h, w, 3), dtype=np.uint8)
        rgb[..., 0] = np.linspace(0, 255, w, dtype=np.uint8)  # R
        rgb[..., 1] = np.linspace(255, 0, w, dtype=np.uint8)  # G
        rgb[..., 2] = 128  # B
        rgb_bytes = rgb.tobytes()
        encoded_rgb_jpeg = ffi.encode_jpeg(rgb_bytes, w, h, 3)
        print(f"JPEG RGB codificado: {len(encoded_rgb_jpeg)} bytes.")
        from io import BytesIO
        img_rgb = Image.open(BytesIO(encoded_rgb_jpeg))
        plt.imshow(img_rgb)
        plt.title("JPEG RGB codificado por FFI")
        plt.axis('off')
        plt.show()
    except Exception as e:
        print(f"Error al codificar imagen RGB: {e}")
else:
    print("Saltando ejemplo RGB (DLL no encontrada)")

## Visual Comparison: jpegexp-rs vs Pillow

### Purpose
Perform a side-by-side visual comparison of images decoded by:
- **Pillow**: Standard Python library
- **jpegexp-rs (FFI)**: Our Rust implementation

### Methodology
1. Generate test image and encode with Pillow
2. Decode the same file with both libraries
3. Display results side-by-side for visual inspection
4. Look for any visual artifacts or differences

In [ ]:
# Visual side-by-side comparison between Pillow and jpegexp-rs (FFI)
# This helps identify any visual differences in decoding quality
if DLL_PATH:
    try:
        # Generar y guardar JPEG con Pillow
        grad = generate_gradient(256, 256)
        img_pillow = Image.fromarray(grad)
        img_pillow.save("grad_pillow_cmp.jpg", format="JPEG")
        # Decodificar con Pillow
        img_loaded = Image.open("grad_pillow_cmp.jpg")
        # Decodificar con FFI
        with open("grad_pillow_cmp.jpg", "rb") as f:
            jpeg_bytes = f.read()
        decoded_pixels, w, h, c = ffi.decode(jpeg_bytes)
        arr_ffi = np.frombuffer(decoded_pixels, dtype=np.uint8).reshape((h, w))
        # Mostrar lado a lado
        fig, axs = plt.subplots(1, 2, figsize=(10, 4))
        axs[0].imshow(img_loaded, cmap='gray')
        axs[0].set_title("Decodificado con Pillow")
        axs[0].axis('off')
        axs[1].imshow(arr_ffi, cmap='gray')
        axs[1].set_title("Decodificado con jpegexp-rs (FFI)")
        axs[1].axis('off')
        plt.show()
    except Exception as e:
        print(f"Error en la comparación visual: {e}")
else:
    print("Saltando comparación visual (DLL no encontrada)")

## Quantitative Comparison: PSNR Analysis

### PSNR (Peak Signal-to-Noise Ratio)
PSNR is a standard metric for measuring image quality and comparing decoders.

### Formula
```
PSNR = 20 * log10(MAX_PIXEL / sqrt(MSE))
```
Where:
- `MAX_PIXEL` = 255 for 8-bit images
- `MSE` = Mean Squared Error between two images

### Interpretation
- **Higher PSNR** = Better quality (less difference)
- **PSNR > 40 dB** = Very good quality
- **PSNR > 30 dB** = Acceptable quality
- **PSNR = ∞** = Perfect match (identical images)

### Purpose
Quantify the difference between Pillow and jpegexp-rs decoding results.

In [ ]:
# Calculate PSNR (Peak Signal-to-Noise Ratio) between two images
# This provides a quantitative measure of image quality differences

def psnr(img1, img2):
    """
    Calculate PSNR between two images.

    Args:
        img1: First image array (numpy array)
        img2: Second image array (numpy array)

    Returns:
        float: PSNR value in decibels (dB), or infinity if images are identical
    """
    mse = np.mean((img1.astype(np.float32) - img2.astype(np.float32)) ** 2)
    if mse == 0:
        return float('inf')
    PIXEL_MAX = 255.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

if DLL_PATH:
    try:
        # Usar los mismos archivos del ejemplo anterior
        grad = generate_gradient(256, 256)
        img_pillow = Image.fromarray(grad)
        img_pillow.save("grad_pillow_cmp.jpg", format="JPEG")
        img_loaded = Image.open("grad_pillow_cmp.jpg")
        arr_pillow = np.array(img_loaded)
        with open("grad_pillow_cmp.jpg", "rb") as f:
            jpeg_bytes = f.read()
        decoded_pixels, w, h, c = ffi.decode(jpeg_bytes)
        arr_ffi = np.frombuffer(decoded_pixels, dtype=np.uint8).reshape((h, w))
        score = psnr(arr_pillow, arr_ffi)
        print(f"PSNR entre Pillow y jpegexp-rs: {score:.2f} dB")
    except Exception as e:
        print(f"Error al calcular PSNR: {e}")
else:
    print("Saltando cálculo de PSNR (DLL no encontrada)")